In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()

In [2]:
df=pd.read_csv('/Users/Ruby/Desktop/TBrain.csv')
df.head()

,device_dev_name,device_family_name,device_hashed_mac,device_os_name,device_type_name,device_vendor_name,event_protocol_id,event_flow_outbound_or_inbound,event_role_device_or_router,event_role_server_or_client,event_rule_category,event_rule_id,event_rule_name,event_rule_reference,event_rule_severity,event_self_ipv4,event_time,router_ip
0,Windows,Computer,c589f89e814a5fda2c5ff061b0e9b283db057b1b,Windows,Desktop/Laptop,Microsoft Corp.,6,outbound,device,client,Access Control,1050015,WEB Cross-site Scripting -34,CVE-2011-2133; CVE-2014-4116,4,192.168.2.243,12/21/2016 3:01:29 AM,207.124.221.172
1,Windows,Computer,c589f89e814a5fda2c5ff061b0e9b283db057b1b,Windows,Desktop/Laptop,Microsoft Corp.,6,outbound,device,client,Access Control,1050015,WEB Cross-site Scripting -34,CVE-2011-2133; CVE-2014-4116,4,192.168.2.243,12/21/2016 3:01:29 AM,207.124.221.172
2,Windows,Computer,c589f89e814a5fda2c5ff061b0e9b283db057b1b,Windows,Desktop/Laptop,Microsoft Corp.,6,outbound,device,client,Access Control,1050015,WEB Cross-site Scripting -34,CVE-2011-2133; CVE-2014-4116,4,192.168.2.243,12/21/2016 3:01:29 AM,207.124.221.172
3,NaN,NaN,2d896d888f3b998e911ee97802daab0dd923e7aa,NaN,NaN,NaN,6,outbound,device,server,Web Attack,1050700,WEB Cross-site Scripting (document.cookie) att...,CVE-2012-0007; CVE-2012-0010; CVE-2012-1861; C...,5,200.223.65.226,12/12/2016 7:05:00 AM,201.97.175.59
4,NaN,NaN,2d896d888f3b998e911ee97802daab0dd923e7aa,NaN,NaN,NaN,6,outbound,device,server,Web Attack,1050700,WEB Cross-site Scripting (document.cookie) att...,CVE-2012-0007; CVE-2012-0010; CVE-2012-1861; C...,5,15.58.51.255,12/12/2016 7:52:24 AM,201.97.175.59


In [3]:
#To drop any rows that have missing data.
df=df.dropna(how='any')

In [4]:
#check isnull
df.isnull().sum() #全為0, ok

device_dev_name                   0
device_family_name                0
device_hashed_mac                 0
device_os_name                    0
device_type_name                  0
device_vendor_name                0
event_protocol_id                 0
event_flow_outbound_or_inbound    0
event_role_device_or_router       0
event_role_server_or_client       0
event_rule_category               0
event_rule_id                     0
event_rule_name                   0
event_rule_reference              0
event_rule_severity               0
event_self_ipv4                   0
event_time                        0
router_ip                         0
dtype: int64

In [5]:
df=pd.DataFrame(df,dtype=np.str)#先變成str

In [6]:
df=df.apply(label.fit_transform)

In [7]:
x=df.drop(['event_rule_name','router_ip','event_time','event_self_ipv4','event_role_device_or_router','device_family_name', 'device_hashed_mac','device_dev_name'],axis=1)

In [8]:
y=df.loc[:,['event_rule_name']]

In [9]:
import xgboost 
from xgboost import XGBClassifier
xgb=XGBClassifier()

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_trainval,X_test,y_trainval,y_test=train_test_split(
x,y,random_state=0)

In [12]:
X_train,X_valid,y_train,y_valid=train_test_split(
X_trainval,y_trainval,random_state=1)

In [13]:
print("Size of train dataset",X_train.shape[0])
print("Size of valid dataset",X_valid.shape[0])
print("Size of test dataset",X_test.shape[0])

Size of train dataset 225365
Size of valid dataset 75122
Size of test dataset 100163


In [14]:
#x_train, x_valid, y_train, y_valid = train_test_split(x, y) 

In [15]:
#print(x_train.shape)  
print(X_valid.shape)  
print(y_train.shape)  
print(y_valid.shape)

(75122, 10)
(225365, 1)
(75122, 1)


In [16]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [18]:
param_grid = {'max_depth': np.arange(3, 10)}
gs = GridSearchCV(DecisionTreeClassifier(), param_grid=param_grid)



In [19]:
gs = gs.fit(X_valid, y_valid)

/Users/Ruby/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/Ruby/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [20]:
print(gs.best_score_)

0.970554564574958


In [21]:
print(gs.best_params_)

{'max_depth': 9}
